# Zadanie dnia

In [11]:
import requests, csv
import pandas as pd
from bs4 import BeautifulSoup

data = requests.get("https://www.mimuw.edu.pl")
soup = BeautifulSoup(data.text, 'html.parser')

events = []

for event in soup.find_all('p', attrs={"date"}):
	parent = event.find_parent()
	akt = dict()
	akt["Date"] = event.text
	akt["Title"] = parent.find_all(('a'))[-1].text
	akt["href"] = "https://www.mimuw.edu.pl" + parent.find_all(('a'))[-1]["href"]
	events.append(akt)
 
csv_file_name = 'events.csv'

field_names = ['Title', 'Date', 'href']

with open(csv_file_name, 'w', newline='') as csv_file:
    csv_writer = csv.DictWriter(csv_file, fieldnames=field_names)
    csv_writer.writeheader()
    csv_writer.writerows(events)
    
df = pd.read_csv(csv_file_name)

sample = df.sample(5)  
print(sample)


                                             Title              Date  \
7                     Sem. Topologia i T. Mnogości  2023-12-20 16:15   
42      Seminarium badawcze „Systemy Inteligentne”  2023-11-24 16:15   
29                             Sem. Topologia Alg.  2023-12-06 10:30   
2   North Atlantic Noncommutative Geometry Seminar  2024-01-24 17:15   
14                                         Sem. RP  2023-12-14 12:15   

                                                 href  
7   https://www.mimuw.edu.pl/seminaria/sem-topolog...  
42  https://www.mimuw.edu.pl/seminaria/seminarium-...  
29  https://www.mimuw.edu.pl/seminaria/sem-topolog...  
2   https://www.mimuw.edu.pl/seminaria/north-atlan...  
14          https://www.mimuw.edu.pl/seminaria/sem-rp  


* Należy pobrać z wykorzystaniem pakietów requests, Beautiful Soup wszystkie wydarzenia zamieszczone w harmonogramie na stronie https://www.mimuw.edu.pl/. 
* Proszę napisać scraper w taki sposób, aby w efekcie działania zapisywał informacje jako lista słowników, których kluczami są tytuł, url do wydarzenia oraz jego data i godzina.
* Otrzymane dane proszę zapisać w formacie csv/xlsx - pomocne może być wykorzystanie ramki danych (pakiet pandas) i metody from_dict.
* Na koniec - prosze o wyświetlenie pięciu losowych wydarzeń z listy pobranych (tytuł, url, data).

,title,href,date
22,Unveiling Microbial Dark Matter: leveraging co...,https://www.mimuw.edu.pl/aktualnosci/seminaria...,2023-12-06 10:15
2,North Atlantic Noncommutative Geometry Seminar,https://www.mimuw.edu.pl/aktualnosci/seminaria...,2024-01-24 17:15
7,Coercive inequalities for Gibbs measures,https://www.mimuw.edu.pl/aktualnosci/seminaria...,2023-12-14 12:15
60,Magdalena Wiertel,https://www.mimuw.edu.pl/doktoraty/magdalena-w...,2023-11-16 16:15
33,Foam evaluation and GL(N)-equivariant Khovanov...,https://www.mimuw.edu.pl/aktualnosci/seminaria...,2023-11-29 10:30


In [4]:
import requests
from random import randint
import json
import os
import logging

#~ Nalezy zmienic API 06.04
#~ https://app.scrapingbee.com/dashboard

SCRAPEOPS_API_KEY = "a863a8aa-6696-474f-8bc9-41c9f752a10a"
SCRAPINGBEE_API_KEY = "HPWIYC12R40AJFXTPHUUTOFNMXMIEBY7YH3YQ63PD8RAR0N1N34UX7N6Z3JIQZBITDUZMJTEEW147TH7"
BASE_PATH = "Items"

def get_headers_list():
  response = requests.get('http://headers.scrapeops.io/v1/browser-headers?api_key=' + SCRAPEOPS_API_KEY)
  json_response = response.json()
  return json_response.get('result', [])

def get_random_header(header_list):
  random_index = randint(0, len(header_list) - 1)
  return header_list[random_index]

def scrapeWebsiteIntoJSON(id, filename = "_tmp", directory = "_tmp"):
    #* Variables
    url = "http://steamcommunity.com/market/itemordershistogram?country=PL&language=polish&currency=6&item_nameid=" + str(id) + "&two_factor=0"
    filename = filename + ".json"
    
    #* Security:
    header_list = get_headers_list()
    
    proxies = \
    {
      "http": "http://" + SCRAPINGBEE_API_KEY + ":render_js=False&premium_proxy=True@proxy.scrapingbee.com:8886",
      "https": "https://" + SCRAPINGBEE_API_KEY + ":render_js=False&premium_proxy=True@proxy.scrapingbee.com:8887"
    }
    
    #* Get source code
    logging.captureWarnings(True)   # hide warnings, because there were a lot XDDD insecure stuff bla bla
    request = requests.get(url=url, proxies=proxies, headers=get_random_header(header_list), verify=False)
    pageSourceCode = request.text

    #* Create folder
    if not os.path.exists(os.path.join(BASE_PATH, directory)):
        os.makedirs(os.path.join(BASE_PATH, directory))

    #* Save into a file
    f = open(os.path.join(BASE_PATH, directory, filename), 'w')
    f.write(pageSourceCode)
    f.close()
    
def getBuyOrderData(filename = "_tmp", directory = "_tmp"):
    #* Variables
    filename = filename + ".json"
    
    #* Create folder
    if not os.path.exists(os.path.join(BASE_PATH, directory)):
        os.makedirs(os.path.join(BASE_PATH, directory))
    
    #* Open file and extract "buy_order_graph"
    f = open(os.path.join(BASE_PATH, directory, filename))
    data = json.load(f)
    buyOrderGraph = data["buy_order_graph"]

    #* Remove useless data
    for i in range(0, len(buyOrderGraph)):
        buyOrderGraph[i] = buyOrderGraph[i][:-1]
        
    return buyOrderGraph
    
def getSellOrderData(filename = "_tmp", directory = "_tmp"):
    #* Variables
    filename = filename + ".json"
    
    #* Create folder
    if not os.path.exists(os.path.join(BASE_PATH, directory)):
        os.makedirs(os.path.join(BASE_PATH, directory))
    
    #* Open file and extract "sell_order_graph"
    f = open(os.path.join(BASE_PATH, directory, filename))
    data = json.load(f)
    sellOrderGraph = data["sell_order_graph"]

    #* Remove useless data
    for i in range(0, len(sellOrderGraph)):
        sellOrderGraph[i] = sellOrderGraph[i][:-1]
        
    return sellOrderGraph

def undoPrefixSum(data):
    cnt = 0
    for i in range(len(data)):
        tmp = data[i][1]
        data[i][1] -= cnt
        cnt = tmp
        
    return data
    

Należy wybrać dowolną stronę internetową (zgodnie z zainteresowaniami), która zezwala na pobieranie danych. Następnie proszę dokonać pobierania wybranych przez Państwa treści. Można spróbować wykorzystac Selenium, chociaż requests + Beautiful Soup są wystarczające. Uzasadnić do czego takie dane mogą się Państwu przydać.  